In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import argparse
import os
from matplotlib import pyplot as plt

import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import log_loss
from decentralized_SGD_logistic import DecentralizedSGDLogistic
from sklearn.linear_model import SGDClassifier
from scipy.special import expit as sigmoid

## RCV dataset from ChocoSGD Paper

In [11]:
def run_logistic(A, y, param):
    m = DecentralizedSGDLogistic(param)
    res = m.fit(A, y)
    print('{} - score: {}'.format(param, m.score(A, y)))
    return res, m

### Load dataset

In [3]:
dataset_path = os.path.expanduser('../ChocoSGD/data/rcv1_test.binary.bz2')
print('Loading dataset...')
A, y = load_svmlight_file(dataset_path)
A_p = A[:100000]
y_p = y[:100000]
y_p = 1*(y_p > 0)

Loading dataset...


### Compute the baseline with SGD

In [5]:
clf = SGDClassifier(loss='log', penalty='l2',alpha =1/A_p.shape[0])
clf.fit(A_p, y_p)
print(clf.score(A_p, y_p))
x_predict = clf.predict(A_p)
exact_optimum = log_loss(x_predict, y_p) + (1 / A_p.shape[0]) * np.sum(x_predict**2)

/home/adirlou/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.97885


### Compute with Decentralized SGD

In [ ]:
reg = 1 / A_p.shape[0]
n_features = A_p.shape[1]

params_disconnected = Parameters(num_epoch=5, lr_type='bottou',
                           initial_lr=10, tau=n_features, regularizer=reg,
                           quantization='full', n_machines=128,
                           method='plain',topology='complete', estimate='final',
                           split_data_random_seed=2, distribute_data=True,
                           split_data_strategy='naive')

res_disconnected, dec_log = run_logistic(A_p, y_p, params_disconnected)

"""params_ring = dict(name="chocosgd-centralized", num_epoch=5, lr_type='bottou',
                           initial_lr=0.2, tau=n_features, regularizer=reg,
                           quantization='full', n_cores=10,
                           method='plain', topology='centralized', estimate='final',
                           split_data_random_seed=2, distribute_data=True,
                           split_data_strategy='naive')

res_centralized = run_logistic(A_p, y_p, params_ring)"""


## Higgs Boson Dataset

In [155]:
def run_logistic(A, y, param):
    m = DecentralizedSGDLogistic(**param)
    list_losses = m.fit(A, y)
    print('{} - score: {}'.format(param, m.score(A, y)))
    return list_losses

In [12]:
def load_csv_data(data_path):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1

    return yb, input_data

In [13]:
def clean(input_data, mean=False):

    #Replace -999 by most frequent value of column
    for i in range(input_data.shape[1]):
        current_col = input_data[:, i]

        if -999.0 in current_col:
            indices_to_change = (current_col == -999.0)
            if mean:
                curr_mean = np.mean(current_col[~indices_to_change])
                current_col[indices_to_change] = curr_mean
            else:
                (values,counts) = np.unique(current_col[~indices_to_change], return_counts=True)
                ind=np.argmax(counts)
                current_col[indices_to_change] = values[ind] if len(values) > 0 else 0

    return input_data

In [14]:
def standardize(x):
    """Standardize the given data"""
    means = x.mean(0)
    stds = x.std(0)
    return (x - means)/stds

### Load dataset

In [163]:
y, A = load_csv_data('train.csv')
A = standardize(clean(A, True))
y = 1 *(y > 0.0)

### Compute the baseline with SGD classifier 

In [164]:
# Fit a SGD
clf = SGDClassifier(loss='log', penalty='l2',alpha =1/A.shape[0], tol=1e-4, n_jobs=1)
clf.fit(A, y)

accuracy = clf.score(A, y)
optimum_x = clf.predict_proba(A)[:, 1]

# Optimal loss, useful for plots
optimum_loss = (-(y.T.dot(np.log(optimum_x)) + (1 - y).T.dot(np.log(1 - optimum_x))) / A.shape[0])

print('Accuracy:', accuracy)
print('Final loss:', optimum_loss)

Accuracy: 0.744164
Final loss: 0.5433024027306941


### Computing with Decentralized SGD

In [159]:
reg = 1 / A.shape[0]
n_features = A.shape[1]

params_disconnected = dict(num_epoch=1, lr_type='bottou',
                           initial_lr=0.1, tau=n_features, regularizer=reg,
                           quantization='full', n_machines=512,
                           method='plain',topology='complete', estimate='final',
                           split_data_random_seed=2, distribute_data=True,
                           split_data_strategy='naive')

list_losses = run_logistic(A, y, params_disconnected)

length of indices: 512
length of last machine indices: 488
current iteration: 0 epoch: 0 epoch_iteration 0 loss 1.6532019533113749 elapsed 0.023987531661987305s
current iteration: 9 epoch: 0 epoch_iteration 9 loss 1.432588552377515 elapsed 0.26162004470825195s
current iteration: 18 epoch: 0 epoch_iteration 18 loss 1.2669745859491743 elapsed 0.48531174659729004s
current iteration: 27 epoch: 0 epoch_iteration 27 loss 1.1404796781369295 elapsed 0.7504069805145264s
current iteration: 36 epoch: 0 epoch_iteration 36 loss 1.0444224931943078 elapsed 0.9808566570281982s
current iteration: 45 epoch: 0 epoch_iteration 45 loss 0.965608208770154 elapsed 1.231112003326416s
current iteration: 54 epoch: 0 epoch_iteration 54 loss 0.9024571572583773 elapsed 1.4517929553985596s
current iteration: 63 epoch: 0 epoch_iteration 63 loss 0.8569154764402902 elapsed 1.6804826259613037s
current iteration: 72 epoch: 0 epoch_iteration 72 loss 0.8216190906627076 elapsed 1.9096367359161377s
current iteration: 81 epoc